# **Introduction to Prompt Engineering**

In [1]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate

# use these for running remote inference at huggingface
!pip install huggingface_hub
!pip install langchain
!pip install langchain-community
!pip install langchain-core

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
from getpass import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass('Enter huggingfacehub api token: ')

Enter huggingfacehub api token:  ········


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

falcon_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

/soft/applications/conda/2024-08-08/mconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-11-23 22:19:13.033074: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-23 22:19:13.052145: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-23 22:19:13.058050: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register facto

RuntimeError: Failed to import transformers.models.falcon.modeling_falcon because of the following error (look up to see its traceback):
/soft/applications/conda/2024-08-08/mconda3/lib/python3.11/site-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE

In [4]:
# load remote model
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.5,
                  "max_length": 128},
)

/var/tmp/pbs.38227.sophia-pbs-01.lab.alcf.anl.gov/ipykernel_704426/454465620.py:5: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  falcon = HuggingFaceHub(


In [5]:
def get_completion_falcon(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from falcon-7b-instruct:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      falcon_response = falcon_pipeline(prompt,
                                      #max_length=500,
                                      do_sample=True,
                                      top_k=10,
                                      num_return_sequences=1,
                                      eos_token_id=tokenizer.eos_token_id,
                                      **kwargs,
                                      )
      response = falcon_response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = falcon.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

### Homework

1. Load in a generative model using the HuggingFace pipeline. Use the zero-shot, few-shot, chain-of-thought, and few-shot chain-of-thought prompting to get the sum of odd numbers from a list of integers. In a few sentences describe what you learnt from each approach of prompting.
- Next, play around with the temperature parameter. In a few sentences describe what you changes you notice.


### a) zero-shot

In [6]:
kwargs = {"max_length": 2200}
zero_shot_prompt =  "What is the sum of the odd numbers in this group: 15, 32, 5, 13, 82, 7, 1."
#15 + 5 + 13 + 7 + 1 = 41

zero_shot_prompt_explanation = get_completion_falcon(zero_shot_prompt, **kwargs)
print(zero_shot_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
What is the sum of the odd numbers in this group: 15, 32, 5, 13, 82, 7, 1.

#### Response from falcon-7b-instruct:

The sum of the odd numbers in the group is -1.

For example, the sum of 15 and 32 is -1, which is -32.

The sum of 5 and 13 is -1, which is -1.

The sum of 82 and 7 is -1, which is -81.

The sum of 1 and 7 is -1, which is -1.

Therefore, the sum of the odd numbers in the group is -1.


### b) few-shot

In [7]:
kwargs = {"max_length": 2200}
few_shot_prompt =  """The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: What is the sum of the odd numbers in this group: 4, 8, 9, 15, 12, 2, 1.
A: The sum of the odd numbers is 25.

Q: What is the sum of the odd numbers in this group: 17, 10, 19, 4, 8, 12, 24.
A: The sum of the odd numbers is 36.

Q: What is the sum of the odd numbers in this group: 16, 11, 14, 4, 8, 13, 24.
A: The sum of the odd numbers is 24.

Q: What is the sum of the odd numbers in this group:: 17, 9, 10, 12, 13, 4, 2.
A: The sum of the odd numbers is 39.

Problem:
Q: What is the sum of the odd numbers in this group: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41

few_shot_prompt_explanation = get_completion_falcon(few_shot_prompt, **kwargs)
print(few_shot_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: What is the sum of the odd numbers in this group: 4, 8, 9, 15, 12, 2, 1.
A: The sum of the odd numbers is 25.

Q: What is the sum of the odd numbers in this group: 17, 10, 19, 4, 8, 12, 24.
A: The sum of the odd numbers is 36.

Q: What is the sum of the odd numbers in this group: 16, 11, 14, 4, 8, 13, 24.
A: The sum of the odd numbers is 24.

Q: What is the sum of the odd numbers in this group:: 17, 9, 10, 12, 13, 4, 2.
A: The sum of the odd numbers is 39.

Problem:
Q: What is the sum of the odd numbers in this group: 15, 32, 5, 13, 82, 7, 1.
A:


#### Response from falcon-7b-instruct:

Q: What is the sum of the odd numbers in this group: 15, 32, 5, 13, 82, 7, 1.
A:

The sum of the odd numbers in this group is 117.


### c) chain-of-thought

In [9]:
kwargs = {"max_length": 2200}
COT_prompt =  "What is the sum of the odd numbers in this group: 15, 32, 5, 13, 82, 7, 1. First state the given numbers. Then select the odd numbers. Add the selected odd numbers. Finish by stating the sum."
#15 + 5 + 13 + 7 + 1 = 41

COT_prompt_explanation = get_completion_falcon(COT_prompt, **kwargs)
print(COT_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
What is the sum of the odd numbers in this group: 15, 32, 5, 13, 82, 7, 1. First state the given numbers. Then select the odd numbers. Add the selected odd numbers. Finish by stating the sum.

#### Response from falcon-7b-instruct: 
The sum of the odd numbers in this group is 13.


### d) few-shot chain-of-thought

In [10]:
kwargs = {"max_length": 2200}
few_shot_COT_prompt =  """The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: What is the sum of the odd numbers in this group: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25.

Q: What is the sum of the odd numbers in this group: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36.

Q: What is the sum of the odd numbers in this group: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24.

Q: What is the sum of the odd numbers in this group:: 17, 9, 10, 12, 13, 4, 2.
A: The given numbers are: (17, 9, 10, 12, 13, 4, 2). Next select the odd numbers (17, 9, 13).  Adding all the selected odd numbers (17, 9, 13) gives 39.

Problem:
Q: What is the sum of the odd numbers in this group: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41

few_shot_COT_prompt_explanation = get_completion_falcon(few_shot_COT_prompt, **kwargs)
print(few_shot_COT_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: What is the sum of the odd numbers in this group: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25.

Q: What is the sum of the odd numbers in this group: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36.

Q: What is the sum of the odd numbers in this group: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24.

Q: What is the sum of the odd numbers in this group:: 17, 9, 10, 12, 13, 4, 2.
A: The given numbe

### e) temperature parameter

In [13]:
kwargs = {"max_length": 2200, "temperature":0.5}
few_shot_COT_prompt =  """The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: What is the sum of the odd numbers in this group: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25.

Q: What is the sum of the odd numbers in this group: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36.

Q: What is the sum of the odd numbers in this group: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24.

Q: What is the sum of the odd numbers in this group:: 17, 9, 10, 12, 13, 4, 2.
A: The given numbers are: (17, 9, 10, 12, 13, 4, 2). Next select the odd numbers (17, 9, 13).  Adding all the selected odd numbers (17, 9, 13) gives 39.

Problem:
Q: What is the sum of the odd numbers in this group: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41

few_shot_COT_prompt_explanation = get_completion_falcon(few_shot_COT_prompt, **kwargs)
print(few_shot_COT_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: What is the sum of the odd numbers in this group: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25.

Q: What is the sum of the odd numbers in this group: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36.

Q: What is the sum of the odd numbers in this group: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24.

Q: What is the sum of the odd numbers in this group:: 17, 9, 10, 12, 13, 4, 2.
A: The given numbe

In [15]:
kwargs = {"max_length": 2200, "temperature":1.2}
few_shot_COT_prompt =  """The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: What is the sum of the odd numbers in this group: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25.

Q: What is the sum of the odd numbers in this group: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36.

Q: What is the sum of the odd numbers in this group: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24.

Q: What is the sum of the odd numbers in this group:: 17, 9, 10, 12, 13, 4, 2.
A: The given numbers are: (17, 9, 10, 12, 13, 4, 2). Next select the odd numbers (17, 9, 13).  Adding all the selected odd numbers (17, 9, 13) gives 39.

Problem:
Q: What is the sum of the odd numbers in this group: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41

few_shot_COT_prompt_explanation = get_completion_falcon(few_shot_COT_prompt, **kwargs)
print(few_shot_COT_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: What is the sum of the odd numbers in this group: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25.

Q: What is the sum of the odd numbers in this group: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36.

Q: What is the sum of the odd numbers in this group: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24.

Q: What is the sum of the odd numbers in this group:: 17, 9, 10, 12, 13, 4, 2.
A: The given numbe

Every method of training gave incorrect answers, but some seemed more incorrect than others.
- zero-shot seemed to be giving random guesses in response to the problem.
- few-shot was guided by the examples in what a response might look like, but was unable to understand the mathematical calculations.
- chain-of-thought, interestingly, ignored my prompt for a multi-part response.
- few-shot chain-of-thought was able to understand what a multi-part response was and how to go about the problem in steps, but was unable to do math.


A low temperature of 0.5 did not seem to greatly affect the output of the model, but a temperature of 1.2 made the model ignore the prompt and give random nonsense as an answer.
